In [1]:
# Ignore this if not running on Google Colab
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.3 MB/s eta 0:00:00


In [31]:
import os
import openai
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff (to overcome rate limit)

import json
import time

import numpy as np

In [6]:
openai.api_key = "REDACTED"

In [7]:
#Exponential backoff decorator
@retry(wait=wait_random_exponential(min=10, max=80), stop=stop_after_attempt(10))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

@retry(wait=wait_random_exponential(min=10, max=80), stop=stop_after_attempt(10))
def create_embeddings(input):
  response = openai.Embedding.create(
      input = input, model = "text-embedding-ada-002"
  )
  return json.dumps(response.data[0]["embedding"])

# Create embeddings for processed docs

In [9]:
rows = []
for filename in os.listdir("docs"):
  contents = open("docs/" + filename).read()
  documents = contents.split("##")
  for document in documents:
    rows.append([filename, document, create_embeddings(document)])

In [10]:
df1 = pd.DataFrame(rows, columns=["filename", "content", "embedding"])
df1.to_csv("docs_embeddings.csv")

# Create embeddings for raw files

In [11]:
main_df = pd.DataFrame()

for filename in os.listdir("raw files"):
  df = pd.read_csv("raw files/" + filename)
  df = df.rename(columns={"Text":"Content", "Summary":"Content"})
  if "Questions" in df.columns:
    df["Content"] = df["Questions"] + "\n" + df["answers"]
    df = df.drop(["Questions", "answers"], axis=1)
  if "Chapter" in df.columns:
    df["Title"] = df["Chapter"] + " " + df["Subheader1"] + " " + df["Subheader2"]
    df = df.drop(["Chapter", "Subheader1", "Subheader2"], axis=1)
  if "Unnamed: 0" in df.columns:
    df = df.drop(["Unnamed: 0"], axis=1)
  main_df = pd.concat([main_df, df])

In [12]:
main_df["embedding"] = main_df["Content"].apply(create_embeddings)

In [13]:
main_df.to_csv("raw_embeddings.csv")

# Find most probable source

In [26]:
doc_df = pd.read_csv("docs_embeddings.csv")
raw_df = pd.read_csv("raw_embeddings.csv")

doc_df["content"] = doc_df["content"].str.strip()
doc_df["embedding"] = doc_df["embedding"].apply(json.loads)
raw_df["embedding"] = raw_df["embedding"].apply(json.loads)

In [41]:
doc_embeddings_full = np.matrix(doc_df["embedding"].to_list())

In [36]:
raw_embeddings_full = np.matrix(raw_df["embedding"].to_list())

In [45]:
similarities = doc_embeddings_full @ raw_embeddings_full.T

In [78]:
best_match_indices = np.argmax(similarities, axis=1)
best_match_indices = [best_match_indices[i,0] for i in range(len(best_match_indices))]

In [80]:
doc_df["Title"] = [raw_df["Title"][i] for i in best_match_indices]

In [87]:
def clean_ema_title(x):
  if "EMA: b'" in x:
    return "EMA: " + x[7:-1]
  return x

doc_df["Title"] = doc_df["Title"].apply(clean_ema_title)

# Merge in URL if missing

In [135]:
raw_df["Title"] = raw_df["Title"].apply(clean_ema_title)
title_map = pd.merge(doc_df["Title"], raw_df[["Title", "URL", "Date"]], how="left").sort_values(["Title", "URL"])

In [136]:
title_map = title_map.drop_duplicates(subset="Title")
# Replaces likely chapters in the Solar PV Handbook with this URL
title_map.loc[title_map["Title"].str.contains("\|") | title_map["Title"].str.contains("An Overview"), "URL"] = "https://www1.bca.gov.sg/docs/default-source/docs-corp-news-and-publications/publications/for-industry/handbook_for_solar_pv_systems_edited_copy.pdf"

<ipython-input-136-10f91095558b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title_map.loc[title_map["Title"].str.contains("\|") | title_map["Title"].str.contains("An Overview"), "URL"] = "https://www1.bca.gov.sg/docs/default-source/docs-corp-news-and-publications/publications/for-industry/handbook_for_solar_pv_systems_edited_copy.pdf"


In [137]:
from bs4 import BeautifulSoup
import requests

def get_url(search):
  url = 'https://www.google.com.sg/search'

  headers = {
    'Accept' : '*/*',
    'Accept-Language': 'en-US,en;q=0.5',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82',
  }
  parameters = {'q': "singapore " + search}

  content = requests.get(url, headers = headers, params = parameters).text
  soup = BeautifulSoup(content, 'html.parser')

  search = soup.find(id = 'search')
  first_link = search.find('a')

  return first_link['href']

In [138]:
title_map.loc[title_map["URL"].isna(), "URL"] = title_map.loc[title_map["URL"].isna(), "Title"].apply(lambda x: get_url(x))

In [139]:
title_map.to_csv("title_maps.csv")

# Merge in URLs

In [142]:
final_df = pd.merge(doc_df, title_map)

In [144]:
with open("ema_knowledge.mdx", "w") as f:
  for _, row in final_df.iterrows():
    if "#" in row["content"]:
      continue
    f.write("## [{}]({})  \n".format(row["Title"], row["URL"]))
    f.write(row["content"])
    f.write("\n\n")